# trAIder - Trading Analysis Exploration

This notebook is used for prototyping and exploring trading strategies.

## Setup

In [ ]:
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime, timedelta

from src.data.data_fetcher import DataFetcher
from src.data.data_preprocessing import DataPreprocessor
from src.indicators.rsi import RSI
from src.indicators.macd import MACD
from src.indicators.cusum import CUSUM
from src.analysis.analysis_engine import AnalysisEngine

%matplotlib inline
plt.style.use('dark_background')

## Data Loading and Preprocessing

In [ ]:
# Initialize components
data_fetcher = DataFetcher()
preprocessor = DataPreprocessor()

# Fetch historical data
symbol = 'BTCUSDT'
interval = '1h'
start_time = datetime.now() - timedelta(days=30)

data = data_fetcher.fetch_historical_data(
    symbol=symbol,
    start_time=start_time,
    interval=interval
)

# Preprocess data
data = preprocessor.clean_data(data)
data = preprocessor.engineer_features(data)

print(f"Loaded {len(data)} data points")
data.head()

## Technical Analysis

In [ ]:
# Initialize analysis engine
engine = AnalysisEngine()

# Add indicators
engine.add_indicator(RSI())
engine.add_indicator(MACD())
engine.add_indicator(CUSUM())

# Run analysis
analyzed_data = engine.analyze(data)

# Display indicator values
analyzed_data[['close', 'rsi', 'macd_line', 'signal_line', 'macd_histogram']].tail()

## Visualization

In [ ]:
# Create candlestick chart
fig = go.Figure()

# Add candlestick
fig.add_trace(
    go.Candlestick(
        x=analyzed_data.index,
        open=analyzed_data['open'],
        high=analyzed_data['high'],
        low=analyzed_data['low'],
        close=analyzed_data['close'],
        name='OHLC'
    )
)

# Update layout
fig.update_layout(
    title=f'{symbol} Price',
    yaxis_title='Price',
    template='plotly_dark'
)

fig.show()

## Signal Analysis

In [ ]:
# Analyze signals
signals = analyzed_data[analyzed_data['composite_signal'] != 0]

print("Signal Distribution:")
print(analyzed_data['composite_signal'].value_counts())
print("\nSignal Timestamps:")
print(signals[['close', 'composite_signal', 'rsi', 'macd_line']].tail())

## Performance Analysis

In [ ]:
# Simple performance simulation
position = 0
entry_price = 0
trades = []

for i in range(1, len(analyzed_data)):
    current_row = analyzed_data.iloc[i]
    signal = current_row['composite_signal']
    
    if position == 0 and signal != 0:
        position = signal
        entry_price = current_row['close']
    elif position != 0 and signal == -position:
        pnl = position * (current_row['close'] - entry_price)
        trades.append({
            'entry_price': entry_price,
            'exit_price': current_row['close'],
            'pnl': pnl,
            'return': pnl / entry_price
        })
        position = 0

# Calculate performance metrics
if trades:
    returns = [t['return'] for t in trades]
    print(f"Number of trades: {len(trades)}")
    print(f"Average return: {np.mean(returns):.2%}")
    print(f"Win rate: {len([r for r in returns if r > 0]) / len(returns):.2%}")
    print(f"Sharpe ratio: {np.mean(returns) / np.std(returns) * np.sqrt(252):.2f}")